## A simple model for demand and supply of geolocalized services in a city

### Load libraries and classes

In [ ]:
import numpy as np
import pandas as pd

import os
import sys
nb_dir = os.path.dirname(os.getcwd()) ## TODO: find way to put this into some global settings
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from references import common_cfg, istat_kpi, city_settings

In [ ]:
from src.models.city_items import AgeGroup, ServiceArea, ServiceType, SummaryNorm # enum classes for the model
from src.models.core import ServiceUnit, ServiceValues, ServiceEvaluator, \
    MappedPositionsFrame, DemandFrame, KPICalculator
from src.models.factories import UnitFactory
from src.models.process_tools import GridMaker, ValuesPlotter, JSONWriter
from src.models.city_items import get_random_pos

In [ ]:
selectedCity = 'Torino'

### Load service data: locations, scales, other info for city

In [ ]:
loaders=UnitFactory.make_loaders_for_city(city_settings.get_city_config(selectedCity))
loaders

In [ ]:
# Initialise with a default lengthscale of 0.5 km
schoolUnits = loaders['Scuole'].load(meanRadius=0.5)

# Initialise with a default lengthscale of 0.3 km
pharmacyUnits = loaders['Farmacie'].load(meanRadius=0.3)

# Initialise with a default lengthscale of 0.3 km
## awful hack to quickly clean tpl file #FIXME
loaders['Fermate TPL']._rawData = loaders['Fermate TPL']._rawData[
    loaders['Fermate TPL']._rawData['route_type'].isin([0,1,3])].reset_index()

tplUnits = loaders['Fermate TPL'].load(meanRadius=0.2)

# Initialise with a default lengthscale of 0.5 km
#libraryUnits = bibliotecheLoader.load(meanRadius=0.8)

### Demand import from Censimento Popolazione e Abitazioni (CPA) 2011

In [ ]:
from scipy.spatial.distance import cdist as eucliDistance
from time import time


In [ ]:
class ServiceEvaluator:
    '''A class to evaluate a given list of service units'''
    
    def __init__(self, unitList, outputServicesIn=[t for t in ServiceType]):
        assert isinstance(unitList, list), 'List expected, got %s' % type(unitList)
        assert all([isinstance(t, ServiceUnit) for t in unitList]), 'ServiceUnits expected in list'
        self.units = unitList
        self.outputServices = outputServicesIn
        self.servicePositions = MappedPositionsFrame(positions=[u.site for u in unitList])

    def evaluate_services_at(self, targetPositions):
        assert isinstance(targetPositions, MappedPositionsFrame), 'Expected MappedPositionsFrame'
        # set all age groups as output default
        outputAgeGroups = AgeGroup.all()
        
        targetsCoordArray = targetPositions[common_cfg.coordColNames].as_matrix()
        targetGeopyArray = targetPositions[common_cfg.positionsCol].values
        # initialise output with dedicated class
        valuesStore = ServiceValues(targetPositions)
                
        # loop over different services
        for thisServType in self.outputServices:
            serviceUnits = [u for u in self.units if u.service == thisServType]
            
            if not serviceUnits:
                continue
            else:
                servicesCoordArray = \
                    self.servicePositions[common_cfg.coordColNames].as_matrix()
                start = time()
                # compute a lower bound for pairwise distances - if this is larger than threshold, set to zero.
                Dmatrix = eucliDistance(servicesCoordArray, targetsCoordArray)*min(common_cfg.approxTileDegToKm)
                print(time() - start)
                
                for thisAgeGroup in outputAgeGroups:
                    if thisAgeGroup in thisServType.demandAges:  # the service can serve this agegroup
                        print('Computing', thisServType, thisAgeGroup)
                        startGroup = time()
                        # each row can be used to drop positions that are too far
                        serviceInteractions = np.zeros(
                            [servicesCoordArray.shape[0], targetsCoordArray.shape[0]])

                        meanVals = []
                        for iUnit in range(len(serviceUnits)):
                            if iUnit%10==0: print('.')
                            thisUnit = serviceUnits[iUnit]
                            # flag the positions that are within the threshold and their values have to be computed
                            bActiveUnit = Dmatrix[iUnit,:] < thisUnit.kerThresholds[thisAgeGroup]
                            serviceInteractions[iUnit, bActiveUnit] = thisUnit.evaluate(
                                targetGeopyArray[bActiveUnit], thisAgeGroup)

                        # aggregate unit contributions according to the service type norm
                        valuesStore[thisServType][thisAgeGroup] = \
                            thisServType.aggregate_units(serviceInteractions, axis=0)
                        print(thisServType, thisAgeGroup, time() - startGroup)
                    else:
                        pass  # leave default value in valuesStore
                        
        return valuesStore

In [ ]:
demandData = DemandFrame.create_from_istat_cpa(selectedCity)

bDemo = True
if bDemo:
    # demo mode, use a portion of the data
    demandTest = DemandFrame(demandData.sample(50).copy(), False)
    testUnits = schoolUnits[::50] + pharmacyUnits[::30] + tplUnits[::30] 
    calculator = KPICalculator(demandTest, testUnits, selectedCity)
else:
    calculator = KPICalculator(demandData, schoolUnits + pharmacyUnits + tplUnits, selectedCity)

In [ ]:
servev = ServiceEvaluator(tplUnits, [ServiceType.TransportStop])
servev.evaluate_services_at(demandData.mappedPositions)

### Evaluate the services offer at the demand points and average over neighbourhood

In [ ]:
# compute and plot demand/supply interaction for localized services 
calculator.evaluate_services_at_demand() # this might take a while.
calculator.compute_kpi_for_localized_services()
pass

### Append Istat KPI and export averaged values by Neighbourhood-Service-AgeGroup to JSON

In [ ]:
# compute istat kpi as well
calculator.compute_kpi_for_istat_values()

# write KPI to json output
jsonMaker = JSONWriter(calculator)
jsonMaker.write_all_files_to_default_path()